<center>

<h1 style="text-align:center"> Pattern Matching </h1>
<h2 style="text-align:center"> CS3100 Fall 2019 </h2>
</center>

## Review


Previously:

* Tuples, Records, Variants
* Polymorphism
* Lists, Option

This lecture:

* Pattern Matching

## Pattern Matching

* Pattern matching is data deconstruction
  + Match on the *shape* of data
  + Extract part(s) of data
  
### Syntax

```ocaml
match e with
| p1 -> e1
| p2 -> e2
...
| pn -> en
```

* p1 ... pn are patterns.

## Pattern Matching on Lists

```
type 'a list = [] | :: of 'a * 'a list
```

* For lists, the patterns allowed follow from the constructors
  + The pattern `[]` matches the value `[]`.
  + The patterh `h::t`
    - matches `2::[]`, binding `h` to `2` and `t` to `[]`.
    - matches `2::3::[]`, binding `h` to `2` and `t` to `3::[]`.
  + The pattern `_` is a **wildcard pattern** and matches anything. 

In [33]:
let list_status l =
  match l with
  | [] -> print_endline "The list is empty"
  | h::t -> Printf.printf "The list is non-empty. Head = %d\n%!" h

val list_status : int list -> unit = <fun>


In [34]:
list_status []

The list is empty


- : unit = ()


In [35]:
list_status [1;2;3]

The list is non-empty. Head = 1


- : unit = ()


In [36]:
list_status (2::[3;4])

The list is non-empty. Head = 2


- : unit = ()


## Why pattern matching is THE GREATEST

1. You cannot forget to match a case (Exhaustivity warning)

In [37]:
let list_status l = 
  match l with 
  | [] -> print_endline "The list is empty"
  | h1::h2::t -> Printf.printf "The list is non-empty. 2nd element = %d\n%!" h2

File "[37]", line 2, characters 2-139:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
_::[]


val list_status : int list -> unit = <fun>


## Why pattern matching is THE GREATEST

1. You cannot forget to match a case (Exhaustivity warning)
2. You cannot duplicate a case (Unused case warning)

In [38]:
let list_status l = 
  match l with 
  | [] -> print_endline "The list is empty"
  | h::t -> Printf.printf "The list is non-empty. Head = %d\n%!" h
  | h1::h2::t -> Printf.printf "The list is non-empty. 2nd element = %d\n%!" h2

File "[38]", line 5, characters 4-13:
Warning 11: this match case is unused.


val list_status : int list -> unit = <fun>


## Why pattern matching is THE GREATEST

1. You cannot forget to match a case (Exhaustivity warning)
2. You cannot duplicate a case (Unused case warning)

<h2> Pattern matching leads to elegant, concise, beautiful code <h2>

## Length of list

In [39]:
let rec length l =
  match l with
  | [] -> 0
  | h::t -> 1 + length t

val length : 'a list -> int = <fun>


What is wrong with this code?

## Length of list (tail recursive)

In [40]:
let rec length' l acc =
  match l with
  | [] -> acc
  | h::t -> length' t (1+acc)
  
let length l = length' l 0

val length' : 'a list -> int -> int = <fun>


val length : 'a list -> int = <fun>


In [41]:
length [1;2;3;4]

- : int = 4


## Match ordering

The patterns are matched in the order that they are written down.

In [42]:
let is_empty l =
  match l with
  | _ -> false
  | [] -> true


File "[42]", line 4, characters 4-6:
Warning 11: this match case is unused.


val is_empty : 'a list -> bool = <fun>


## nth 

Implement indexing into the list

In [43]:
let rec nth l n = 
  match (l, n) with
  | (hd::_, 0) -> Some hd
  | (hd::tl, n) -> nth tl (n-1)
  | _ -> None

val nth : 'a list -> int -> 'a option = <fun>


In [44]:
nth [1;2;3] 4

- : int option = None


## reverse

Implement the reverse of a list.

In [45]:
let rec rev_list l acc = 
  match l with
  | [] -> acc
  | hd::tl -> rev_list tl (hd::acc)

let rev_list l = rev_list l []

val rev_list : 'a list -> 'a list -> 'a list = <fun>


val rev_list : 'a list -> 'a list = <fun>


In [46]:
assert (rev_list [1;2;3] = [3;2;1])

- : unit = ()


## append

Implement the append of two lists.

In [47]:
[1;2;3] @ [4]

- : int list = [1; 2; 3; 4]


In [48]:
let rec append l1 l2 = 
  match l1 with
  | [] -> l2
  | hd::tl -> hd::(append tl l2)

val append : 'a list -> 'a list -> 'a list = <fun>


In [49]:
assert (append [1;2;3] [4;5;6] = [1;2;3;4;5;6])

- : unit = ()


## Nested Matching



In [2]:
type color = Red | Green | Blue

type point = {x : int; y : int}

type shape = 
  | Circle of point * float (* center, radius *)
  | Rect of point * point   (* lower-left, upper-right *)
  | ColorPoint of point * color

type color = Red | Green | Blue


type point = { x : int; y : int; }


type shape =
    Circle of point * float
  | Rect of point * point
  | ColorPoint of point * color


## Nested Matching

Is the first shape in a list of shapes a red point?

In [3]:
let is_hd_red_circle l = 
  match l with
  | ColorPoint(_,Red)::_ -> true
  | _ -> false

val is_hd_red_circle : shape list -> bool = <fun>


## Nested Matching

Print the coordinates if the point is green.

In [4]:
let rec print_green_point l =
  match l with
  | [] -> ()
  | ColorPoint({x;y}, Green)::tl -> 
      Printf.printf "x = %d y = %d\n%!" x y;
      print_green_point tl
  | _::tl -> print_green_point tl

val print_green_point : shape list -> unit = <fun>


In [5]:
print_green_point [Rect ({x=1;y=1},{x=2;y=2});
                   ColorPoint ({x=0;y=0}, Green);
                   Circle ({x=1;y=3}, 5.4);
                   ColorPoint ({x=4;y=6}, Green)]

x = 0 y = 0
x = 4 y = 6


- : unit = ()


## When do you use ";"

When you evaluate an expression just the effect, you can sequence the expression with a semi-colon.

```ocaml
let () = print_endline "Hello, world!" in
e
```

is equivalent to:

```ocaml
print_endline "Hello, world!";
e
```

Latter is considered better style.

## Exceptions

* OCaml has support for exceptions. 
  + Similar to the ones found in C++ & Java.
* Exceptions are (mostly) just variants.

```ocaml
type exn
exception MyException of string
```

* The type `exn` is an **built-in extensible variant type**.
  + New constructors of this type can be added after its original declaration.
* Exceptions are raised with `raise e` where `e` is of type `exn`.
* Handling exceptions is similar to pattern matching.

## Find the green point

Given a list of shapes return a point whose colour is green. Otherwise, raise `NoGreenPoint` exception.

In [6]:
exception NoGreenPoint

let rec find_green_point l = 
  match l with
  | [] -> raise NoGreenPoint
  | h::tl ->
    match h with
    | ColorPoint (p, Green) -> p
    | _ -> find_green_point tl

exception NoGreenPoint


val find_green_point : shape list -> point = <fun>


## Find the green point

In [7]:
find_green_point []

error: runtime_error

In [8]:
find_green_point [Rect ({x=1;y=1},{x=2;y=2}); ColorPoint ({x=0;y=0}, Green)]

- : point = {x = 0; y = 0}


## Handling the exception

Given a list of shapes return `Some p` where `p` is a green point. Otherwise, return `None`.

In [9]:
let find_green_point_opt l =
  try Some (find_green_point l) with
  | NoGreenPoint -> None

val find_green_point_opt : shape list -> point option = <fun>


In [10]:
find_green_point_opt []

- : point option = None


In [11]:
find_green_point_opt [Rect ({x=1;y=1},{x=2;y=2}); ColorPoint ({x=0;y=0}, Green)]

- : point option = Some {x = 0; y = 0}


## Exceptions: Recommendations

* Avoid exceptions in your code.
  + Unhandled exceptions are runtime errors; aim to avoid this.
* No exhaustiveness check for exceptions (why?).
* Whenever you might need to use exceptions, think whether you can replace that with

```ocaml
type 'a option = None | Some of 'a
```

or 

```ocaml
type ('a,'b) result = Ok of 'a | Error of 'b
```

## Exercise 

`List.hd : 'a list -> 'a` and `List.tl: 'a list -> 'a list` are functions from the [list standard library](https://caml.inria.fr/pub/docs/manual-ocaml/libref/List.html). They raise exception when the given list is empty. Implement safe versions of the functions whose signatures are:

In [60]:
let safe_hd (l : 'a list) : 'a option = failwith "not implemented"
let safe_tl (l : 'a list) : 'a list option = failwith "not implemented"

val safe_hd : 'a list -> 'a option = <fun>


val safe_tl : 'a list -> 'a list option = <fun>


<center>

<h1 style="text-align:center"> Fin. </h1>
</center>